# eICU Data Aggregation

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [32]:
# Read data files for patient and admissions data
patient = pd.read_csv('./eicu-research-database/patient.csv')
admission = pd.read_csv('./eicu-research-database/admissionDx.csv')

We want to predict the occurrence of mortality and length of stay for ICU patients with respiratory diseases so we will filter for only respiratory patients in the admissions table and merge it with the patient table.

In [33]:
# Filter for respiratory patients in the patients table
resp_admission = admission.loc[admission.admitdxname == 'Respiratory', :]
resp_patient_ids = resp_admission.patientunitstayid

patient = patient.loc[patient.patientunitstayid.isin(resp_patient_ids), :]

print(patient.shape)
print(patient.uniquepid.nunique())

(25813, 29)
21528


In [34]:
pd.set_option('display.max_columns', None)
patient.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeyear,hospitaldischargetime24,hospitaldischargeoffset,hospitaldischargelocation,hospitaldischargestatus,unittype,unitadmittime24,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
6,141203,128948,Female,77,Caucasian,66,90,"Arrest, respiratory (without cardiac arrest)",160.0,22:23:00,-1336,Floor,2014,16:22:00,4063,Other External,Alive,Med-Surg ICU,20:39:00,Floor,1,admit,70.2,70.3,03:48:00,1869,Floor,Alive,002-23234
12,141260,128995,Female,43,African American,73,99,Asthma,172.7,01:57:00,-18,Emergency Department,2015,09:34:00,1879,Other,Alive,Med-Surg ICU,02:15:00,Emergency Department,1,admit,69.9,70.3,20:08:00,1073,Floor,Alive,002-55885
17,141276,129009,Female,59,Caucasian,63,95,"Arrest, respiratory (without cardiac arrest)",165.1,14:43:00,-1,Operating Room,2015,19:20:00,1716,Home,Alive,Med-Surg ICU,14:44:00,Direct Admit,1,admit,156.6,156.6,18:48:00,1684,Home,Alive,002-9090
20,141289,129020,Female,61,Caucasian,67,109,Pneumothorax,162.6,17:32:00,-7593,Emergency Department,2014,21:28:00,1283,Other Hospital,Alive,Med-Surg ICU,00:05:00,Floor,2,readmit,NaN,NaN,21:28:00,1283,Other Hospital,Alive,002-67735
28,141337,129059,Female,72,Hispanic,71,113,Thoracotomy for other reasons,154.9,20:27:12,-7,Operating Room,2015,18:02:00,2728,Home,Alive,Med-Surg ICU,20:34:00,Operating Room,1,admit,67.8,68.5,15:40:00,1146,Floor,Alive,002-48469


### Create total length of stay variable

In [35]:
patient['icuduration'] = (
    patient.hospitaldischargeoffset - patient.hospitaladmitoffset
)
patient.drop(columns=['hospitaldischargeoffset', 'hospitaladmitoffset', 
                      'hospitaldischargeyear', 'unitdischargeoffset'], 
             inplace=True)

# Get duration in days
patient.icuduration = patient.icuduration / 1440

### Create death indicator

In [36]:
patient['died'] = np.nan
patient.loc[(patient.hospitaldischargestatus == 'Expired'), 
            'died'] = 1
patient.loc[(patient.hospitaldischargestatus == 'Alive'), 
            'died'] = 0
patient.drop(columns=['hospitaldischargestatus', 'unitdischargestatus'], 
             inplace=True)

In [37]:
# Drop columns that may cause data leakage in predicting mortality
patient.drop(columns=['hospitaldischargelocation', 'unitdischargelocation'],
             inplace=True)

### Create variable to calculate difference in admission and discharge weight

In [38]:
patient['weightdiffafterdischarge'] = (
    patient.dischargeweight - patient.admissionweight
)

We know that there is multiple records (admissions) for some patients. For simplicity, we will only consider the first stay for each patient. 

In [39]:
patient = patient.loc[patient.unitvisitnumber == 1, :]
patient.drop(columns='unitvisitnumber', inplace=True)

print(patient.shape)
print(patient.uniquepid.nunique())

(21007, 23)
18802


### Drop duplicate records/admissions for patients

In [40]:
duplicates = patient.loc[patient.duplicated(['uniquepid'], keep=False), :]
pids_to_drop = duplicates.uniquepid.unique().tolist()

patient = patient.loc[~(patient.uniquepid.isin(pids_to_drop)), :]
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 23)
17167


### Merge APACHE data to patient table

In [41]:
# Load data on APACHE APS variables
apache_aps_vars = pd.read_csv('./eicu-research-database/apacheApsVar.csv')
selected_cols = ['patientunitstayid', 'intubated', 'vent', 'dialysis', 'urine',
                 'wbc', 'temperature', 'respiratoryrate', 'sodium', 'heartrate', 
                 'meanbp', 'ph', 'creatinine', 'albumin', 'glucose', 'bilirubin', 
                 'fio2', 'pao2', 'pco2', 'bun']
apache_aps_vars = apache_aps_vars.loc[:, selected_cols]

print(apache_aps_vars.shape)
print(apache_aps_vars.patientunitstayid.nunique())

(171177, 20)
171177


In [42]:
patient = patient.merge(apache_aps_vars, how='left', on='patientunitstayid')
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 42)
17167


### Merge APACHE patient results data to patient table

In [43]:
apache_patient_result = (
    pd.read_csv('./eicu-research-database/apachePatientResult.csv')
)
selected_cols = ['patientunitstayid', 'apachescore', 'predictedicumortality', 
                 'actualicumortality', 'predictediculos', 'actualiculos', 
                 'actualhospitalmortality', 'predictedhospitallos', 
                 'unabridgedactualventdays', 'unabridgedunitlos']
apache_patient_result = apache_patient_result.loc[:, selected_cols]

print(apache_patient_result.shape)
print(apache_patient_result.patientunitstayid.nunique())

(297064, 10)
148532


Since there are duplicate records for `patientunitstayid`, we will aggregate the data by this key.

In [44]:
apache_patient_result_agg = (
    apache_patient_result.groupby('patientunitstayid')
                         .agg(meanapachescore = ('apachescore', 'mean'),
                              meanpredictedicumortality = ('predictedicumortality', 
                                                           'mean'), 
                              meanpredictediculos = ('predictediculos', 'mean'),
                              meanventdays = ('unabridgedactualventdays', 'mean')
                             )
                         .reset_index()
)

patient = patient.merge(apache_patient_result_agg, how='left', 
                        on='patientunitstayid')

print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 46)
17167


### Merge data of variables from APACHE predictions to patient table

In [45]:
apache_pred_var = pd.read_csv('./eicu-research-database/apachePredVar.csv')
print(apache_pred_var.shape)
print(apache_pred_var.patientunitstayid.nunique())

selected_cols = ['patientunitstayid', 'aids', 'lymphoma', 'immunosuppression',
                 'diabetes']
apache_pred_var = apache_pred_var.loc[:, selected_cols]

(171177, 51)
171177


In [46]:
patient = patient.merge(apache_pred_var, how='left', on='patientunitstayid')
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 50)
17167


### Merge hospital data to patient table

In [47]:
hospital = pd.read_csv('./eicu-research-database/hospital.csv')
hospital.rename(columns = {'region':'hospitalregion'}, inplace = True)
print(hospital.shape)
print(hospital.hospitalid.nunique())

patient = patient.merge(hospital, how='left', on='hospitalid')
print(patient.shape)
print(patient.uniquepid.nunique())

(208, 4)
208
(17167, 53)
17167


### Merge drug data to patient table

In [48]:
drug = pd.read_csv('./eicu-research-database/infusionDrug.csv', 
                   low_memory=False)
print(drug.shape)
print(drug.patientunitstayid.nunique())

(4803719, 9)
73547


There are duplicate values for each patient which would suggest that some of the patients were given multiple drugs during their visit. So, we must aggregate the data for each patient visit. However, the drug amount measurements are measured differently based on the drug, therefore, we will standardize these variables so that all measurements are on the same scale.

In [49]:
selected_cols = ['infusionrate', 'drugamount', 'volumeoffluid']
temp = drug.loc[:, selected_cols]

# Standardize infusionrate, drugamount, and volumneoffluid
temp_scaled = StandardScaler().fit_transform(temp)
drugs_scaled = pd.DataFrame(temp_scaled, index=temp.index, columns=temp.columns)
drugs_scaled['patientunitstayid'] = drug['patientunitstayid']

In [50]:
drug_agg = (
    drugs_scaled.groupby('patientunitstayid')
                .agg(numberofinfusions = ('patientunitstayid', 'size'),
                     mininfusionrate = ('infusionrate', 'min'),
                     meaninfusionrate = ('infusionrate', 'mean'),
                     maxinfusionrate = ('infusionrate', 'max'),
                     mindrugamount = ('drugamount', 'min'),
                     meandrugamount = ('drugamount', 'mean'),
                     maxdrugamount = ('drugamount', 'max'),
                     minvolumeoffluid = ('volumeoffluid', 'min'),
                     meanvolumeoffluid = ('volumeoffluid', 'mean'),
                     maxvolumeoffluid = ('volumeoffluid', 'max')) 
                .reset_index()
)

patient = patient.merge(drug_agg, how='left', on='patientunitstayid')
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 63)
17167


### Merge lab test data to patient table

In [51]:
lab = pd.read_csv('./eicu-research-database/timeserieslab.csv')
print(lab.shape)
print(lab.patientunitstayid.nunique())

(6814790, 4)
88664


In [52]:
selected_cols = ['patientunitstayid', 'labname', 'labresult']
lab = lab.loc[:, selected_cols]

lab_agg = (
    lab.groupby(['patientunitstayid', 'labname'])
       .agg(meanlabresult = ('labresult', 'mean'))
       .reset_index()
)

lab_agg['labname'] = lab_agg['labname'].replace({'-basos': 'basos', 
                                                 '-eos': 'eos', 
                                                 '-lymphs': 'lymphs', 
                                                 '-monos': 'monos', 
                                                 '-polys': 'polys'})

lab_wide = (
    pd.pivot(lab_agg, 
             index=['patientunitstayid'], 
             columns = 'labname',
             values = 'meanlabresult').add_prefix('lab_mean_').reset_index()
)

patient = patient.merge(lab_wide, how='left', on='patientunitstayid')
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 109)
17167


### Merge periodic vital sign data with patient table

In [53]:
vital_periodic = (
    pd.read_csv('./eicu-research-database/timeseries_readyformodel.csv')
)
print(vital_periodic.shape)
print(vital_periodic.patientunitstayid.nunique())

(250920, 38)
10455


In [54]:
vital_periodic.columns

Index(['patient', 'time', 'FiO2', 'bedside glucose', 'cvp', 'heartrate',
       'noninvasivediastolic', 'noninvasivemean', 'noninvasivesystolic',
       'respiration', 'sao2', 'st1', 'st2', 'st3', 'systemicdiastolic',
       'systemicmean', 'systemicsystolic', 'temperature', 'FiO2_mask',
       'bedside glucose_mask', 'cvp_mask', 'heartrate_mask',
       'noninvasivediastolic_mask', 'noninvasivemean_mask',
       'noninvasivesystolic_mask', 'respiration_mask', 'sao2_mask', 'st1_mask',
       'st2_mask', 'st3_mask', 'systemicdiastolic_mask', 'systemicmean_mask',
       'systemicsystolic_mask', 'temperature_mask', 'hour',
       'patientunitstayid', 'died', 'urine'],
      dtype='object')

In [55]:
vital_agg = (
    vital_periodic.groupby('patient')
                  .agg(minFi02 = ('FiO2', 'min'),
                       meanFi02 = ('FiO2', 'mean'),
                       maxFi02 = ('FiO2', 'max'),
                       minbedsideglucose = ('bedside glucose', 'min'),
                       meanbedsideglucose = ('bedside glucose', 'mean'),
                       maxbedsideglucose = ('bedside glucose', 'max'),
                       mincvp = ('cvp', 'min'),
                       meancvp = ('cvp', 'mean'),
                       maxcvp = ('cvp', 'max'),
                       minheartrate = ('heartrate', 'min'),
                       meanheartrate = ('heartrate', 'mean'),
                       maxheartrate = ('heartrate', 'max'),
                       minnoninvasivediastolic = ('noninvasivediastolic', 'min'),
                       meannoninvasivediastolic = ('noninvasivediastolic', 'mean'),
                       maxnoninvasivediastolic = ('noninvasivediastolic', 'max'),
                       minnoninvasivemean = ('noninvasivemean', 'min'),
                       meannoninvasivemean = ('noninvasivemean', 'mean'),
                       maxnoninvasivemean = ('noninvasivemean', 'max'),
                       minnoninvasivesystolic = ('noninvasivesystolic', 'min'),
                       meannoninvasivesystolic = ('noninvasivesystolic', 'mean'),
                       maxnoninvasivesystolic = ('noninvasivesystolic', 'max'),
                       minrespiration = ('respiration', 'min'),
                       meanrespiration = ('respiration', 'mean'),
                       maxrespiration = ('respiration', 'max'),
                       minsao2 = ('sao2', 'min'),
                       meansao2 = ('sao2', 'mean'),
                       maxsao2 = ('sao2', 'max'),
                       minst1 = ('st1', 'min'),
                       meanst1 = ('st1', 'mean'),
                       maxst1 = ('st1', 'max'),
                       minst2 = ('st2', 'min'),
                       meanst2 = ('st2', 'mean'),
                       maxst2 = ('st2', 'max'),
                       minst3 = ('st3', 'min'),
                       meanst3 = ('st3', 'mean'),
                       maxst3 = ('st3', 'max'),
                       minsystemicdiastolic = ('systemicdiastolic', 'min'),
                       meansystemicdiastolic = ('systemicdiastolic', 'mean'),
                       maxsystemicdiastolic = ('systemicdiastolic', 'max'),
                       minsystemicmean = ('systemicmean', 'min'),
                       meansystemicmean = ('systemicmean', 'mean'),
                       maxsystemicmean = ('systemicmean', 'max'),
                       minsystemicsystolic = ('systemicsystolic', 'min'),
                       meansystemicsystolic = ('systemicsystolic', 'mean'),
                       maxsystemicsystolic = ('systemicsystolic', 'max'),
                       mintemperature = ('temperature', 'min'),
                       meantemperature = ('temperature', 'mean'),
                       maxtemperature = ('temperature', 'max'))
                  .reset_index()
)

patient = patient.merge(vital_agg, how='left', left_on='patientunitstayid', 
                        right_on='patient')
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 158)
17167


### Merge respiratory charting data with patient table

In [56]:
resp = pd.read_csv('./eicu-research-database/timeseriesresp.csv', 
                   low_memory=False)
print(resp.shape)
print(resp.patientunitstayid.nunique())

(2308984, 4)
52166


In [57]:
# Clean up respchartvalue column given that it is a mixed data type
resp['respchartvalue'] = resp.respchartvalue.str.extract(r'(\d+)', expand=False)
resp.respchartvalue = resp.respchartvalue.astype(float)

In [58]:
selected_cols = ['patientunitstayid', 'respchartvaluelabel', 'respchartvalue']
resp = resp.loc[:, selected_cols]

resp_agg = (
    resp.groupby(['patientunitstayid', 'respchartvaluelabel'])
        .agg(meanrespchartvalue = ('respchartvalue', 'mean'))
        .reset_index()
)

resp_wide = (
    pd.pivot(resp_agg, 
             index=['patientunitstayid'], 
             columns = 'respchartvaluelabel',
             values = 'meanrespchartvalue').add_prefix('resp_mean_').reset_index()
)

patient = patient.merge(resp_wide, how='left', on='patientunitstayid')
print(patient.shape)
print(patient.uniquepid.nunique())

(17167, 171)
17167


### Merge aperiodic vital sign data with patient table

In [59]:
vital_aperiodic = pd.read_csv('./eicu-research-database/vitalAperiodic.csv')
print(vital_aperiodic.shape)
print(vital_aperiodic.patientunitstayid.nunique())

(25075074, 13)
189753


In [60]:
# Aggregate vital sign data for each patient stay
vital_agg = (
    vital_aperiodic.groupby('patientunitstayid')
                   .agg(minpaop = ('paop', 'min'),
                        meanpaop = ('paop', 'mean'),
                        maxpaop = ('paop', 'max'),
                        mincardiacoutput = ('cardiacoutput', 'min'),
                        meancardiacoutput = ('cardiacoutput', 'mean'),
                        maxcardiacoutput = ('cardiacoutput', 'max'),
                        mincardiacinput = ('cardiacinput', 'min'),
                        meancardiacinput = ('cardiacinput', 'mean'), 
                        maxcardiacinput = ('cardiacinput', 'max'),
                        minsvr = ('svr', 'min'),
                        meansvr = ('svr', 'mean'),
                        maxsvr = ('svr', 'max'),
                        minsvri = ('svri', 'min'),
                        meansvri = ('svri', 'mean'),
                        maxsvri = ('svri', 'max'),
                        minpvr = ('pvr', 'min'),
                        meanpvr = ('pvr', 'mean'),
                        maxpvr = ('pvr', 'max'),
                        minpvri = ('pvri', 'min'),
                        meanpvri = ('pvri', 'mean'),
                        maxpvri = ('pvri', 'max')) 
                   .add_prefix('aperiodic_')
                   .reset_index()
)

patient = patient.merge(vital_agg, how='left', on='patientunitstayid')
print(patient.shape)
print(patient.uniquepid.nunique())

# Save aggregated patient table to csv
patient.to_csv('raw_agg_patient_table.csv', index=False)

(17167, 192)
17167
